In [14]:
import pandas as pd
import numpy as np

In [3]:
import warnings

In [4]:
warnings.filterwarnings('ignore', message='Workbook contains no default style')

In [5]:
df3 = pd.read_excel("C:/Users/user/Documents/서울시_포폴/서울시공실률/매장용빌딩.xlsx", engine="openpyxl", header=[0, 1, 2])

In [6]:
df3

지역별(1) 지역별(2)   2023 1/4                                  2023 2/4          \
   지역별(1) 지역별(2) 임대료 (천원/㎡) 공실률 (%) 수익률 정보 (%)             임대료 (천원/㎡) 공실률 (%)   
   지역별(1) 지역별(2)         소계      소계      투자수익률 소득수익률 자본수익률         소계      소계   
0      서울     소계       52.2     8.6       0.66  0.67 -0.01       52.2     8.4   
1    도심지역     소계       84.3      16       0.54  0.69 -0.15       83.7    15.3   
2     NaN    광화문       86.6    16.6       0.71  0.82 -0.11       86.3      18   
3     NaN    남대문       74.7     9.7        0.3  0.69 -0.39       68.6     9.7   
4     NaN    동대문       44.7    13.5       0.21  0.76 -0.56       45.5    12.7   
..    ...    ...        ...     ...        ...   ...   ...        ...     ...   
68    NaN    장안동       36.2     7.5        0.6  0.79 -0.19       36.3     7.2   
69    NaN     천호         47     4.7       0.79  0.77  0.02       47.1     6.5   
70    NaN    청량리       45.1      10        0.4  0.61 -0.21       45.2     9.9   
71    NaN    혜화동       61.8     7.3       0.63  0.66 -0.03       62.3     7.3   
72    NaN     화곡       35.3    14.4       0.68   0.8 -0.12       35.2    11.8   

               ...       2023                                      2024  \
   수익률 정보 (%)  ... 임대료 (천원/㎡) 공실률 (%) 수익률 정보 (%)             임대료 (천원/㎡)   
        투자수익률  ...         소계      소계      투자수익률 소득수익률 자본수익률         소계   
0        0.75  ...       52.6     8.4       3.56  2.52  1.01       54.9   
1        0.71  ...       84.1    14.1       3.55  2.64  0.89       86.4   
2        0.83  ...       86.4    13.5       3.87  3.08  0.77       91.4   
3        0.54  ...         69    10.1       2.93  2.58  0.34       83.2   
4        0.72  ...       45.6    12.1       3.28  2.96  0.31       49.8   
..        ...  ...        ...     ...        ...   ...   ...        ...   
68       0.63  ...       36.6     7.2       3.08     3  0.08       37.3   
69       0.95  ...       47.2     4.3       4.35  2.83  1.49       46.8   
70       0.39  ...       45.3     7.8        2.8  2.34  0.46       45.7   
71       0.76  ...       62.9     5.9       3.63  2.41   1.2       67.7   
72       0.57  ...       35.2    12.7       2.75  3.09 -0.34       35.5   

                                   
   공실률 (%) 수익률 정보 (%)              
        소계      투자수익률 소득수익률 자본수익률  
0      8.9       5.52  2.47     3  
1     12.3       5.57  2.71  2.81  
2     14.8       5.96  3.19   2.7  
3     15.0       5.36  2.58  2.73  
4     14.9       5.31  2.89  2.38  
..     ...        ...   ...   ...  
68     8.1       4.41   2.8  1.58  
69     7.4       5.71  2.82  2.83  
70    10.0       5.06  2.33  2.68  
71     5.6       6.84  2.56   4.2  
72     9.4       3.77  3.17  0.58  

[73 rows x 52 columns]

In [ ]:
# 멀티 인덱스 → 평면화 → (필수 최소 전처리) → DB 저장

# 멀티 인덱스 작업은 “DB 저장 전에 반드시” 해야 한다 !

In [22]:
# ✔ 저장 전에 해야 하는 작업 (공실률 기준)

# MultiIndex → 컬럼 평면화
# → 내가 이미 한 new_columns 작업 👍

# 상위 지역 ffill (지역별1)
# → 키 보존을 위한 필수 작업

# wide(가로) 형태 DataFrame을 long(세로) 형태로 변환
# → 스키마 안정화 (가장 중요)

# 숫자형 변환 (value)

# 👉 이 4개는 **“전처리”가 아니라 “구조 정규화”**라서
# DB 저장 전에 함 ! 

In [7]:
# MultiIndex → 컬럼 평면화

new_columns = []
for col in df3.columns.values:
    if col[0] == '지역별(1)' and col[1] == '지역별(1)':
        new_columns.append('지역별1')
    elif col[0] == '지역별(2)' and col[1] == '지역별(2)':
        new_columns.append('지역별2')
    else:
        # 기간_지표_세부항목 형태로
        new_columns.append('_'.join(col).strip())

df3.columns = new_columns
print(df3.columns.tolist())

['지역별1', '지역별2', '2023 1/4_임대료 (천원/㎡)_소계', '2023 1/4_공실률 (%)_소계', '2023 1/4_수익률 정보 (%)_투자수익률', '2023 1/4_수익률 정보 (%)_소득수익률', '2023 1/4_수익률 정보 (%)_자본수익률', '2023 2/4_임대료 (천원/㎡)_소계', '2023 2/4_공실률 (%)_소계', '2023 2/4_수익률 정보 (%)_투자수익률', '2023 2/4_수익률 정보 (%)_소득수익률', '2023 2/4_수익률 정보 (%)_자본수익률', '2023 3/4_임대료 (천원/㎡)_소계', '2023 3/4_공실률 (%)_소계', '2023 3/4_수익률 정보 (%)_투자수익률', '2023 3/4_수익률 정보 (%)_소득수익률', '2023 3/4_수익률 정보 (%)_자본수익률', '2023 4/4_임대료 (천원/㎡)_소계', '2023 4/4_공실률 (%)_소계', '2023 4/4_수익률 정보 (%)_투자수익률', '2023 4/4_수익률 정보 (%)_소득수익률', '2023 4/4_수익률 정보 (%)_자본수익률', '2024 1/4_임대료 (천원/㎡)_소계', '2024 1/4_공실률 (%)_소계', '2024 1/4_수익률 정보 (%)_투자수익률', '2024 1/4_수익률 정보 (%)_소득수익률', '2024 1/4_수익률 정보 (%)_자본수익률', '2024 2/4_임대료 (천원/㎡)_소계', '2024 2/4_공실률 (%)_소계', '2024 2/4_수익률 정보 (%)_투자수익률', '2024 2/4_수익률 정보 (%)_소득수익률', '2024 2/4_수익률 정보 (%)_자본수익률', '2024 3/4_임대료 (천원/㎡)_소계', '2024 3/4_공실률 (%)_소계', '2024 3/4_수익률 정보 (%)_투자수익률', '2024 3/4_수익률 정보 (%)_소득수익률', '2024 3/4_수익률 정보 (%)_자본수익률', '2024 4/4_임대료 (천원/㎡)_소계', '2024

In [9]:
df3.head(3)

,지역별1,지역별2,2023 1/4_임대료 (천원/㎡)_소계,2023 1/4_공실률 (%)_소계,2023 1/4_수익률 정보 (%)_투자수익률,2023 1/4_수익률 정보 (%)_소득수익률,2023 1/4_수익률 정보 (%)_자본수익률,2023 2/4_임대료 (천원/㎡)_소계,2023 2/4_공실률 (%)_소계,2023 2/4_수익률 정보 (%)_투자수익률,...,2023_임대료 (천원/㎡)_소계,2023_공실률 (%)_소계,2023_수익률 정보 (%)_투자수익률,2023_수익률 정보 (%)_소득수익률,2023_수익률 정보 (%)_자본수익률,2024_임대료 (천원/㎡)_소계,2024_공실률 (%)_소계,2024_수익률 정보 (%)_투자수익률,2024_수익률 정보 (%)_소득수익률,2024_수익률 정보 (%)_자본수익률
0,서울,소계,52.2,8.6,0.66,0.67,-0.01,52.2,8.4,0.75,...,52.6,8.4,3.56,2.52,1.01,54.9,8.9,5.52,2.47,3
1,도심지역,소계,84.3,16,0.54,0.69,-0.15,83.7,15.3,0.71,...,84.1,14.1,3.55,2.64,0.89,86.4,12.3,5.57,2.71,2.81
2,NaN,광화문,86.6,16.6,0.71,0.82,-0.11,86.3,18,0.83,...,86.4,13.5,3.87,3.08,0.77,91.4,14.8,5.96,3.19,2.7


In [10]:
# 상위 지역 ffill (지역별1)

df3["지역별1"] = df3["지역별1"].ffill()

In [11]:
df3

,지역별1,지역별2,2023 1/4_임대료 (천원/㎡)_소계,2023 1/4_공실률 (%)_소계,2023 1/4_수익률 정보 (%)_투자수익률,2023 1/4_수익률 정보 (%)_소득수익률,2023 1/4_수익률 정보 (%)_자본수익률,2023 2/4_임대료 (천원/㎡)_소계,2023 2/4_공실률 (%)_소계,2023 2/4_수익률 정보 (%)_투자수익률,...,2023_임대료 (천원/㎡)_소계,2023_공실률 (%)_소계,2023_수익률 정보 (%)_투자수익률,2023_수익률 정보 (%)_소득수익률,2023_수익률 정보 (%)_자본수익률,2024_임대료 (천원/㎡)_소계,2024_공실률 (%)_소계,2024_수익률 정보 (%)_투자수익률,2024_수익률 정보 (%)_소득수익률,2024_수익률 정보 (%)_자본수익률
0,서울,소계,52.2,8.6,0.66,0.67,-0.01,52.2,8.4,0.75,...,52.6,8.4,3.56,2.52,1.01,54.9,8.9,5.52,2.47,3
1,도심지역,소계,84.3,16,0.54,0.69,-0.15,83.7,15.3,0.71,...,84.1,14.1,3.55,2.64,0.89,86.4,12.3,5.57,2.71,2.81
2,도심지역,광화문,86.6,16.6,0.71,0.82,-0.11,86.3,18,0.83,...,86.4,13.5,3.87,3.08,0.77,91.4,14.8,5.96,3.19,2.7
3,도심지역,남대문,74.7,9.7,0.3,0.69,-0.39,68.6,9.7,0.54,...,69,10.1,2.93,2.58,0.34,83.2,15.0,5.36,2.58,2.73
4,도심지역,동대문,44.7,13.5,0.21,0.76,-0.56,45.5,12.7,0.72,...,45.6,12.1,3.28,2.96,0.31,49.8,14.9,5.31,2.89,2.38
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
68,기타지역,장안동,36.2,7.5,0.6,0.79,-0.19,36.3,7.2,0.63,...,36.6,7.2,3.08,3,0.08,37.3,8.1,4.41,2.8,1.58
69,기타지역,천호,47,4.7,0.79,0.77,0.02,47.1,6.5,0.95,...,47.2,4.3,4.35,2.83,1.49,46.8,7.4,5.71,2.82,2.83
70,기타지역,청량리,45.1,10,0.4,0.61,-0.21,45.2,9.9,0.39,...,45.3,7.8,2.8,2.34,0.46,45.7,10.0,5.06,2.33,2.68
71,기타지역,혜화동,61.8,7.3,0.63,0.66,-0.03,62.3,7.3,0.76,...,62.9,5.9,3.63,2.41,1.2,67.7,5.6,6.84,2.56,4.2


## (1) 공실률 wide(가로) 형태 DataFrame을 long(세로) 형태로 변환

In [19]:
import pandas as pd
import numpy as np

def to_vacancy_long(df3):
    """
    공실률 wide(가로) 형태 DataFrame을
    long(세로) 형태로 변환
    """

    # 식별자 컬럼
    id_vars = ["지역별1", "지역별2"]

    # 녹일 컬럼
    value_vars = [c for c in df3.columns if c not in id_vars]

    # wide → long
    long = df3.melt(
        id_vars=id_vars,
        value_vars=value_vars,
        var_name="col",
        value_name="value"
    )

    # col 예시: "2023 1/4_공실률 (%)_소계"
    parts = long["col"].str.split("_", n=2, expand=True)

    long["period"] = parts[0].str.strip()
    long["metric"] = parts[1].str.strip()
    long["metric_detail"] = parts[2].str.strip()

    long = long.drop(columns=["col"])

    # 숫자형 정리
    long["value"] = (
        long["value"]
        .astype(str)
        .str.replace(",", "", regex=False)
        .str.strip()
    )
    long["value"] = pd.to_numeric(long["value"], errors="coerce")

    # 지역 컬럼 공백 제거
    long["지역별1"] = long["지역별1"].astype(str).str.strip()
    long["지역별2"] = long["지역별2"].astype(str).str.strip()

    # 핵심 키 NULL 제거 (안전장치)
    long = long.dropna(
        subset=["지역별1", "지역별2", "period", "metric", "metric_detail"]
    )

    return long


In [20]:
long = to_vacancy_long(df3)

In [21]:
long

,지역별1,지역별2,value,period,metric,metric_detail
0,서울,소계,52.20,2023 1/4,임대료 (천원/㎡),소계
1,도심지역,소계,84.30,2023 1/4,임대료 (천원/㎡),소계
2,도심지역,광화문,86.60,2023 1/4,임대료 (천원/㎡),소계
3,도심지역,남대문,74.70,2023 1/4,임대료 (천원/㎡),소계
4,도심지역,동대문,44.70,2023 1/4,임대료 (천원/㎡),소계
...,...,...,...,...,...,...
3645,기타지역,장안동,1.58,2024,수익률 정보 (%),자본수익률
3646,기타지역,천호,2.83,2024,수익률 정보 (%),자본수익률
3647,기타지역,청량리,2.68,2024,수익률 정보 (%),자본수익률
3648,기타지역,혜화동,4.20,2024,수익률 정보 (%),자본수익률


## (2) DB 에 적재

In [18]:
import os
from pathlib import Path
import pandas as pd
import numpy as np
import pymysql

# =========================
# ✅ 설정
# =========================
ROOT_DIR = r"C:\Users\user\Documents\서울시_포폴\서울시공실률"
TABLE = "stg_seoul_vacancy_long"

MYSQL = dict(
    host="localhost",
    user="root",
    password="********",
    db="seoul",
    port=3306,
    charset="utf8mb4",
    autocommit=False
)

CHUNK_SIZE = 20000


# =========================
# 1) long 변환
# =========================
def to_vacancy_long(df3: pd.DataFrame) -> pd.DataFrame:
    id_vars = ["지역별1", "지역별2"]
    value_vars = [c for c in df3.columns if c not in id_vars]

    long = df3.melt(
        id_vars=id_vars,
        value_vars=value_vars,
        var_name="col",
        value_name="value"
    )

    # col 예: "2023 1/4_공실률 (%)_소계"
    parts = long["col"].str.split("_", n=2, expand=True)
    long["period"] = parts[0].astype(str).str.strip()
    long["metric"] = parts[1].astype(str).str.strip()
    long["metric_detail"] = parts[2].astype(str).str.strip()

    long = long.drop(columns=["col"])

    # 값 숫자화
    long["value"] = (long["value"].astype(str)
                     .str.replace(",", "", regex=False)
                     .str.strip())
    long["value"] = pd.to_numeric(long["value"], errors="coerce")

    # 공백 정리
    long["지역별1"] = long["지역별1"].astype(str).str.strip()
    long["지역별2"] = long["지역별2"].astype(str).str.strip()

    # 핵심 키 결측 제거
    long = long.dropna(subset=["지역별1", "지역별2", "period", "metric", "metric_detail"])

    return long


# =========================
# 2) MultiIndex 컬럼 평면화 (지역별/구분별 둘 다 지원)
# =========================
def flatten_columns(df: pd.DataFrame) -> pd.DataFrame:
    """
    MultiIndex(3레벨)인 경우:
    ('지역별(1)',...) 또는 ('구분별(1)',...) -> '지역별1'
    ('지역별(2)',...) 또는 ('구분별(2)',...) -> '지역별2'
    나머지 -> '기간_지표_세부'
    """
    if not isinstance(df.columns, pd.MultiIndex):
        return df

    new_cols = []
    for col in df.columns.values:
        lvl0 = str(col[0]).strip()

        if lvl0 in ("지역별(1)", "구분별(1)"):
            new_cols.append("지역별1")
        elif lvl0 in ("지역별(2)", "구분별(2)"):
            new_cols.append("지역별2")
        else:
            new_cols.append("_".join([str(x).strip() for x in col]))

    df = df.copy()
    df.columns = new_cols
    return df


# =========================
# 3) 파일 읽기 (엑셀, 헤더 3줄 가정)
# =========================
def read_file(path: str) -> pd.DataFrame:
    # 네가 보여준 MultiIndex가 3레벨이라 header=[0,1,2]로 가정
    return pd.read_excel(path, header=[0, 1, 2])


# =========================
# 4) DB insert
# =========================
def insert_long(conn, long_df: pd.DataFrame):
    cols = ["building_type", "지역별1", "지역별2", "period", "metric", "metric_detail", "value", "source_file"]

    sql = f"""
    INSERT IGNORE INTO {TABLE}
    (`building_type`,`지역별1`,`지역별2`,`period`,`metric`,`metric_detail`,`value`,`source_file`)
    VALUES (%s,%s,%s,%s,%s,%s,%s,%s)
    """

    sub = long_df[cols].astype(object).where(pd.notnull(long_df[cols]), None)
    data = [tuple(r) for r in sub.to_numpy()]

    with conn.cursor() as cur:
        for i in range(0, len(data), CHUNK_SIZE):
            cur.executemany(sql, data[i:i+CHUNK_SIZE])
            print(f"  inserted {min(i+CHUNK_SIZE, len(data))}/{len(data)}")

    conn.commit()


# =========================
# 5) main
# =========================
def main():
    conn = pymysql.connect(**MYSQL)

    targets = ["매장용빌딩", "오피스빌딩"]
    files = []
    for t in targets:
        matched = sorted(Path(ROOT_DIR).glob(f"{t}.*"))
        if not matched:
            conn.close()
            raise ValueError(f"파일을 못 찾았어: {t}.*  (경로: {ROOT_DIR})")
        files.append(matched[0])

    for f in files:
        building_type = f.stem  # '매장용빌딩' / '오피스빌딩'
        print(f"\n[START] {f.name}  building_type={building_type}")

        df = read_file(str(f))
        df = flatten_columns(df)

        # ✅ 컬럼 존재 체크
        if "지역별1" not in df.columns or "지역별2" not in df.columns:
            print("현재 컬럼 일부:", list(df.columns)[:10])
            conn.close()
            raise ValueError(f"{f.name}에서 지역별1/지역별2 컬럼 생성 실패. 헤더 구조 확인 필요.")

        # ✅ 상위 지역 채우기 (도심지역 -> 광화문/남대문/동대문 ...)
        df["지역별1"] = (df["지역별1"].astype(str)
                         .str.strip()
                         .replace({"nan": np.nan, "": np.nan})
                         .ffill())

        df["지역별2"] = (df["지역별2"].astype(str)
                         .str.strip()
                         .replace({"nan": np.nan, "": np.nan}))

        # long 변환
        long = to_vacancy_long(df)

        # 메타 추가
        long["building_type"] = building_type
        long["source_file"] = f.name

        # insert
        insert_long(conn, long)

        print(f"[DONE] {f.name} long_rows={len(long)}")

    conn.close()
    print("\nALL DONE")


if __name__ == "__main__":
    main()



[START] 매장용빌딩.xlsx  building_type=매장용빌딩
  inserted 3650/3650
[DONE] 매장용빌딩.xlsx long_rows=3650

[START] 오피스빌딩.xlsx  building_type=오피스빌딩
  inserted 1530/1530
[DONE] 오피스빌딩.xlsx long_rows=1530

ALL DONE
